<a href="https://colab.research.google.com/github/subhamchatterjee21/NETFLIX-clone/blob/main/Audio_similarity_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [3]:
from os.path import join
import librosa
from IPython.display import Audio

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install openl3


In [ ]:
import librosa
import openl3
import faiss
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

# Load the audio file
file_path_bn= "/content/drive/MyDrive/bn/cv-corpus-17.0-delta-2024-03-15/bn/clips/common_voice_bn_39620358.mp3"
audio, sr = librosa.core.load(file_path_bn, sr=44100, mono=True)

file_path_as= "/content/drive/MyDrive/Ass/cv-corpus-16.1-2023-12-06/as/clips/common_voice_as_20870771.mp3"
audio_as, sr = librosa.core.load(file_path_as, sr=44100, mono=True)



In [ ]:
# Bengali
plt.figure(figsize=(10, 4))
librosa.display.waveshow(audio, sr=sr)
plt.title('Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()
print("")
display(Audio(audio, rate=3000))

In [ ]:

# Assamese
plt.figure(figsize=(10, 4))
librosa.display.waveshow(audio_as, sr=sr)
plt.title('Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.show()
print("")
display(Audio(audio_as, rate=sr))

In [ ]:
open_emb_bn, ts_bn = openl3.get_audio_embedding(audio, sr, input_repr="mel128", frontend='librosa')
open_emb_as, ts_as = openl3.get_audio_embedding(audio_as, sr, input_repr="mel128", frontend='librosa')

In [ ]:
open_emb_bn.shape, open_emb_bn.mean(axis=0).shape, open_emb_bn.mean(axis=0)[:20]

In [ ]:
open_emb_as.shape, open_emb_as.mean(axis=0).shape, open_emb_as.mean(axis=0)[:20]

In [ ]:
# Normalize embeddings
open_emb_bn /= np.linalg.norm(open_emb_bn, axis=1, keepdims=True)
open_emb_as /= np.linalg.norm(open_emb_as, axis=1, keepdims=True)


In [ ]:
# Build FAISS index for Assamese embeddings
index_as = faiss.IndexFlatIP(open_emb_as.shape[1])
index_as.add(open_emb_as)

In [ ]:
# Define Bengali embeddings as query
query_embedding_bn = open_emb_bn.mean(axis=0)  # Using mean embedding of Bengali audio as query
query_embedding_bn /= np.linalg.norm(query_embedding_bn)  # Normalize query embedding

In [ ]:
# Perform search using Bengali query on Assamese embeddings index
k = 2  # Number of nearest neighbors to retrieve
distances, indices = index_as.search(np.expand_dims(query_embedding_bn, axis=0), k)

In [ ]:
from IPython.display import Audio

# Print results
print("Top {} most similar Assamese embeddings to Bengali query:".format(k))
for i, (distance, index) in enumerate(zip(distances[0], indices[0])):
    print("Rank {}: Distance = {:.4f}, Index = {}".format(i+1, distance, index))
    # Load and play similar Assamese audio corresponding to the top-ranked embeddings
    similar_audio_as, _ = librosa.core.load(file_path_as, sr=sr, mono=True)
    print("Playing similar Assamese audio for rank {}...".format(i+1))
    display(Audio(similar_audio_as, rate=sr))